In [ ]:
import os, re, html, pandas as pd, stanza, pathlib, torch
from tqdm import tqdm
from stanza.utils.conll import CoNLL

In [3]:
BASE_DIR = pathlib.Path('dataset')
FILES = [
    # BASE_DIR / 'A1.csv',
    # BASE_DIR / 'B2.csv',
    # BASE_DIR / 'C2.csv',
    BASE_DIR / 'cefr_leveled_texts.csv',
]
LANG  = "en"
dfs = []
for f in FILES:
    if os.path.exists(f):
        dfs.append(pd.read_csv(f, encoding="utf-8"))
if not dfs:
    raise FileNotFoundError("None of the CSVs were found. Check FILES paths.")
data = pd.concat(dfs, ignore_index=True)
if "text" not in data.columns:
    raise ValueError("CSV must contain a 'text' column.")
data = data.dropna(subset=["text"]).reset_index(drop=True)

In [9]:
#Optional for testing
# data = data.head(200)
levels = data.label.unique()

In [11]:
stanza.download(LANG, processors="tokenize,mwt,pos,lemma,depparse", verbose=False)
USE_GPU = False
try:
    import torch
    USE_GPU = torch.cuda.is_available()
except Exception:
    pass
nlp = stanza.Pipeline(LANG, processors="tokenize,mwt,pos,lemma,depparse",use_gpu=USE_GPU, verbose=False)

In [19]:
os.makedirs("dataset/tagged-stanza", exist_ok=True)

In [20]:
for l in levels:
    cefr = data[data["label"]==l]
    TEMPLATE = "dataset/tagged-stanza/{}_{:04d}.conllu"
    for i, txt in tqdm(enumerate(cefr["text"].astype(str), start=1), total=len(data)):
        doc = nlp(txt)
        # This writes a NEW file per document (row)
        CoNLL.write_doc2conll(doc, TEMPLATE.format(l, i))

 14%|█████████████████▋                                                                                                                 | 202/1494 [03:42<23:43,  1.10s/it]


In [10]:
levels

array(['B2', 'A2', 'C1', 'B1', 'A1', 'C2'], dtype=object)